# Quickstart Tutorial: Ceramic Armor Discovery Framework

This tutorial provides a quick introduction to the Ceramic Armor Discovery Framework.

## What You'll Learn
- How to set up the environment
- Basic screening workflow
- ML model training and prediction
- Results visualization

## Prerequisites
- Python 3.11+
- Materials Project API key (set in .env file)
- PostgreSQL database running

## 1. Setup and Configuration

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from ceramic_discovery.config import config
from ceramic_discovery.dft.materials_project_client import MaterialsProjectClient
from ceramic_discovery.ceramics.ceramic_system import CeramicSystem
from ceramic_discovery.screening.screening_engine import ScreeningEngine

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Imports successful")
print(f"✓ Configuration loaded from: {config.hdf5.data_path}")

## 2. Explore Base Ceramic Systems

Let's examine the baseline properties of our ceramic systems.

In [ ]:
# Initialize ceramic system
ceramic = CeramicSystem()

# Get baseline properties for SiC
sic_props = ceramic.get_baseline_properties('SiC')

print("SiC Baseline Properties:")
print(f"  Hardness: {sic_props['hardness']:.1f} GPa")
print(f"  Fracture Toughness: {sic_props['fracture_toughness']:.1f} MPa·m^0.5")
print(f"  Density: {sic_props['density']:.2f} g/cm³")
print(f"  Thermal Conductivity (1000°C): {sic_props['thermal_conductivity_1000C']:.1f} W/m·K")

## 3. Screen Dopant Candidates

Screen potential dopants for improved ballistic performance.

In [ ]:
# Initialize screening engine
engine = ScreeningEngine()

# Screen dopants for SiC
dopants = ['Ti', 'Zr', 'Hf']
concentrations = [0.01, 0.02, 0.05]  # 1%, 2%, 5% atomic

print(f"Screening {len(dopants)} dopants at {len(concentrations)} concentrations...")

results = engine.screen_dopants(
    base_system='SiC',
    dopants=dopants,
    concentrations=concentrations,
    stability_threshold=0.1  # ΔE_hull ≤ 0.1 eV/atom
)

print(f"\n✓ Screening complete")
print(f"  Total candidates: {results['total_screened']}")
print(f"  Viable candidates: {results['viable_count']}")

## 4. Visualize Screening Results

In [ ]:
# Convert results to DataFrame
df_results = pd.DataFrame(results['candidates'])

# Plot stability vs predicted performance
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Stability distribution
ax1.hist(df_results['energy_above_hull'], bins=20, edgecolor='black')
ax1.axvline(0.1, color='red', linestyle='--', label='Stability threshold')
ax1.set_xlabel('Energy Above Hull (eV/atom)')
ax1.set_ylabel('Count')
ax1.set_title('Thermodynamic Stability Distribution')
ax1.legend()

# Performance ranking
top_10 = df_results.nsmallest(10, 'energy_above_hull')
ax2.barh(range(len(top_10)), top_10['predicted_v50'])
ax2.set_yticks(range(len(top_10)))
ax2.set_yticklabels(top_10['composition'])
ax2.set_xlabel('Predicted V50 (m/s)')
ax2.set_title('Top 10 Candidates by Stability')

plt.tight_layout()
plt.show()

## 5. Train ML Model

Train a machine learning model to predict ballistic performance.

In [ ]:
from ceramic_discovery.ml.model_trainer import ModelTrainer

# Prepare training data
X = df_results[['hardness', 'fracture_toughness', 'density', 'thermal_conductivity']]
y = df_results['experimental_v50']  # If available

# Initialize trainer
trainer = ModelTrainer(model_type='ensemble', cv_folds=5)

# Train model
print("Training ML model...")
metrics = trainer.train(X, y)

print(f"\n✓ Training complete")
print(f"  R² score: {metrics['r2_score']:.3f}")
print(f"  RMSE: {metrics['rmse']:.2f} m/s")
print(f"  Target range: {config.ml.target_r2_min:.2f} - {config.ml.target_r2_max:.2f}")

## 6. Make Predictions with Uncertainty

Use the trained model to make predictions with uncertainty quantification.

In [ ]:
from ceramic_discovery.ballistics.ballistic_predictor import BallisticPredictor

# Initialize predictor
predictor = BallisticPredictor()
predictor.load_trained_model(trainer.model)

# Make predictions with uncertainty
predictions = predictor.predict_with_uncertainty(X)

# Visualize predictions
fig, ax = plt.subplots(figsize=(10, 6))

ax.errorbar(
    range(len(predictions)),
    predictions['mean'],
    yerr=[predictions['lower_ci'], predictions['upper_ci']],
    fmt='o',
    capsize=5,
    label='Predictions with 95% CI'
)

ax.set_xlabel('Material Index')
ax.set_ylabel('Predicted V50 (m/s)')
ax.set_title('Ballistic Performance Predictions with Uncertainty')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Export Results

Save results for further analysis or reporting.

In [ ]:
from ceramic_discovery.reporting.data_exporter import DataExporter

# Initialize exporter
exporter = DataExporter()

# Export to multiple formats
exporter.export_to_csv(df_results, './results/screening_results.csv')
exporter.export_to_json(results, './results/screening_results.json')

print("✓ Results exported successfully")
print("  CSV: ./results/screening_results.csv")
print("  JSON: ./results/screening_results.json")

## Next Steps

- Explore `01_exploratory_analysis.ipynb` for detailed property analysis
- See `02_ml_model_training.ipynb` for advanced ML techniques
- Check `03_dopant_screening.ipynb` for comprehensive screening workflows
- Review `04_mechanism_analysis.ipynb` for understanding performance drivers